In [37]:
# Imports
import numpy as np
import tensorflow.compat.v1 as tf
import tensorflow.compat.v1.keras as K
tf.compat.v1.disable_eager_execution()
tf.disable_v2_behavior()

In [38]:
# 0. Sequential
"""
Write a function that builds a neural network with the Keras library
Use Sequential model since that's in the task name
"""
def build_model_0(nx, layers, activations, lambtha, keep_prob):
    """
    nx: number of input features to the network
    layers: list containing the number of nodes in each layer of the network
    activations: list containing the activation functions for each layer
    lambtha: L2 regularization parameter
    keep_prob: probability that a node will be kept for dropout

    Not allowed to use `Input` class
    Returns the keras model
    """
    model = K.Sequential()
    # Create our input layer
    model.add(K.layers.InputLayer(input_shape=(nx,)))
    # Create all of our hidden layers
    for layer in range(len(layers)):
        model.add(K.layers.Dense(units=layers[layer],
                                 activation=activations[layer],
                                 kernel_regularizer=K.regularizers.l2(lambtha)
                                 ))
        # Handle dropout on hidden layers
        if layer < len(layers) - 1:
            model.add(K.layers.Dropout(1 - keep_prob))
    return model

In [39]:
# 0-main
network = build_model_0(784, [256, 256, 10], ['tanh', 'tanh', 'softmax'], 0.001, 0.95)
network.summary()
print(network.losses)

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_26 (Dense)            (None, 256)               200960    
                                                                 
 dropout_16 (Dropout)        (None, 256)               0         
                                                                 
 dense_27 (Dense)            (None, 256)               65792     
                                                                 
 dropout_17 (Dropout)        (None, 256)               0         
                                                                 
 dense_28 (Dense)            (None, 10)                2570      
                                                                 
Total params: 269,322
Trainable params: 269,322
Non-trainable params: 0
_________________________________________________________________
[<tf.Tensor 'dense_26/kernel/Regularizer/mul:0' 

In [40]:
# 0. Sequential | Attempt 2
"""
Write a function that builds a neural network with the Keras library
Use Sequential model since that's in the task name
"""
def build_model_1(nx, layers, activations, lambtha, keep_prob):
    """
    nx: number of input features to the network
    layers: list containing the number of nodes in each layer of the network
    activations: list containing the activation functions for each layer
    lambtha: L2 regularization parameter
    keep_prob: probability that a node will be kept for dropout

    Not allowed to use `Input` class
    Returns the keras model
    """
    model = K.Sequential()
    # Create our input layer
    model.add(K.layers.Dense(units=layers[0],
                             activation = activations[0],
                             kernel_regularizer=K.regularizers.l2(lambtha),
                             input_shape=(nx,)))
    # Create all of our hidden layers
    for layer in range(1, len(layers)):
        if layer <= len(layers) - 1:
            model.add(K.layers.Dropout(1 - keep_prob))
        model.add(K.layers.Dense(units=layers[layer],
                                 activation=activations[layer],
                                 kernel_regularizer=K.regularizers.l2(lambtha)
                                 ))
        # Handle dropout on hidden layers
    return model

In [41]:
# 0-main | Attempt 2
network = build_model_1(784, [256, 256, 10], ['tanh', 'tanh', 'softmax'], 0.001, 0.95)
network.summary()
print(network.losses)

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_29 (Dense)            (None, 256)               200960    
                                                                 
 dropout_18 (Dropout)        (None, 256)               0         
                                                                 
 dense_30 (Dense)            (None, 256)               65792     
                                                                 
 dropout_19 (Dropout)        (None, 256)               0         
                                                                 
 dense_31 (Dense)            (None, 10)                2570      
                                                                 
Total params: 269,322
Trainable params: 269,322
Non-trainable params: 0
_________________________________________________________________
[<tf.Tensor 'dense_29/kernel/Regularizer/mul:0' 

In [42]:
# Task 1. Input
"""
Write a function that builds a neural network with the Keras library
"""
def build_model(nx, layers, activations, lambtha, keep_prob):
    """
    nx: number of input features to the network
    layers: list containing the number of nodes in each layer of the network
    activations: list containing the activation functions for each layer
    lambtha: L2 regularization parameter
    keep_prob: probability that a node will be kept for dropout

    Not allowed to use `Sequential` class
    Returns the keras model
    """
    inputs = K.Input(shape=(nx,))
    x = K.layers.Dense(units=layers[0],
                       activation=activations[0],
                       kernel_regularizer=K.regularizers.l2(lambtha)
                       )(inputs)
    for layer in range(1, len(layers)):
        dropout_layer = K.layers.Dropout(1 - keep_prob)(x)
        x = K.layers.Dense(units=layers[layer],
                           activation=activations[layer],
                           kernel_regularizer=K.regularizers.l2(lambtha)
                           )(dropout_layer)

    model = K.Model(inputs=inputs, outputs=x)
    return model



In [43]:
# 1-main
model = build_model(200, [10], ['tanh'], 0.01, 0.6)
model.summary()

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_10 (InputLayer)       [(None, 200)]             0         
                                                                 
 dense_32 (Dense)            (None, 10)                2010      
                                                                 
Total params: 2,010
Trainable params: 2,010
Non-trainable params: 0
_________________________________________________________________


In [44]:
# 2. Optimize
"""
Write a function that sets up Adam optimization for a keras model with
categorical crossentropy loss and accuracy metrics
"""
def optimize_model(network, alpha, beta1, beta2):
    """
    network: the model to optimize
    alpha: learning rate
    beta1: the first Adam optimization parameter
    beta2: second Adam optimization parameter
    Returns none
    """

    Adam_opt = K.optimizers.Adam(lr=alpha,
                                 beta_1=beta1,
                                 beta_2=beta2)
    network.compile(optimizer=Adam_opt,
                    metrics=['accuracy'],
                    loss="categorical_crossentropy")
    return None

In [45]:
# 2-main
# model = build_model(784, [256, 256, 10], ['tanh', 'tanh', 'softmax'], 0.001, 0.95)
# optimize_model(model, 0.01, 0.99, 0.9)
# print(model.loss)
# print(model.metrics)
# opt = model.optimizer
# print(opt.__class__)
# with tf.Session() as sess:
#     sess.run(tf.global_variables_initializer())
#     print(sess.run((opt.lr, opt.beta_1, opt.beta_2))) 

In [46]:
# 3. One Hot
"""
Runction that converts a label vector into a one-hot matrix
"""
def one_hot(labels, classes=None):
    """
    labels: labels for dat
    """
    return K.utils.to_categorical(labels, classes)


In [47]:
# 3-main
labels = np.load('../data/MNIST.npz')['Y_train'][:10]
print(labels)
print(one_hot(labels))   

[5 0 4 1 9 2 1 3 1 4]
[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]]


In [48]:
# Task 4. Train
"""
Write a function that trains a model using mini-batch gradient descent
"""
def train_model(network, data, labels, batch_size,
                epochs, verbose=True, shuffle=False):
    """
    network: model to train
    data: numpy.ndarray of shape (m, nx) containing the input data
        m: number of data points
        nx: number of features
    labels: one-hot numpy.ndarray of shape (m, classes) with the data labels
        m: number of data points
        classes: number of classes
    batch_size: size of the batch used for mini-batch gradient descent
    epochs: number of passes through data for mini-batch gradient descent
    verbose: boolean that determines if the output should be printed during
        training
    shuffle: boolean that determines whether to shuffle the batches every
        epoch. Normally, it is a good idea to shuffle, but for reproducibility
        we have chosen to set the default to False
    Returns: the History object generated after training the model
    """
    return network.fit(x=data,
                       y=labels,
                       batch_size=batch_size,
                       epochs=epochs,
                       verbose=verbose,
                       shuffle=shuffle,
                       )

In [50]:
# 4-main
# Force Seed - fix for Keras
SEED = 0

import os
os.environ['PYTHONHASHSEED'] = str(SEED)
import random
random.seed(SEED)
import numpy as np
np.random.seed(SEED)
# import tensorflow as tf
tf.set_random_seed(SEED)
# import tensorflow.keras as K
session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.backend.set_session(sess)

datasets = np.load('../data/MNIST.npz')
X_train = datasets['X_train']
X_train = X_train.reshape(X_train.shape[0], -1)
Y_train = datasets['Y_train']
Y_train_oh = one_hot(Y_train)

lambtha = 0.0001
keep_prob = 0.95
network = build_model(784, [256, 256, 10], ['relu', 'relu', 'softmax'], lambtha, keep_prob)
alpha = 0.001
beta1 = 0.9
beta2 = 0.999
optimize_model(network, alpha, beta1, beta2)
batch_size = 64
epochs = 5
train_model(network, X_train, Y_train_oh, batch_size, epochs)

Train on 50000 samples
Epoch 1/5
   64/50000 [..............................] - ETA: 56s - loss: 2.3745 - acc: 0.0312

2023-03-03 15:16:00.922657: W tensorflow/c/c_api.cc:291] Operation '{name:'training_2/Adam/dense_38/kernel/v/Assign' id:2622 op device:{requested: '', assigned: ''} def:{{{node training_2/Adam/dense_38/kernel/v/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](training_2/Adam/dense_38/kernel/v, training_2/Adam/dense_38/kernel/v/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


50000/50000 [==============================] - 3s 61us/sample - loss: 0.3278 - acc: 0.9200
Epoch 2/5
50000/50000 [==============================] - 3s 62us/sample - loss: 0.1742 - acc: 0.9650
Epoch 3/5
50000/50000 [==============================] - 3s 61us/sample - loss: 0.1420 - acc: 0.9751
Epoch 4/5
50000/50000 [==============================] - 3s 69us/sample - loss: 0.1242 - acc: 0.9808
Epoch 5/5
50000/50000 [==============================] - 3s 53us/sample - loss: 0.1155 - acc: 0.9835
